In [1]:
"""
    MinMax Best
"""

import random
from typing import Any

import chess
from chess import Board

import minmax_heuristic
from minmax_heuristic import weight


class MyBestHeuristic:
    move: Any

    def __init__(self):
        pass

    def __repr__(self):
        pass

    # metric of evaluation
    def H_0(self, state: Board):
        pieces = state.piece_map()
        ret = 0
        for index in pieces:
            if pieces[index].color:
                ret += weight(pieces[index].piece_type)
            else:
                ret -= weight(pieces[index].piece_type)
        return ret

    # minimax best (best subnodes)
    def H_L(self, state: Board, l: int):
        if l == 0:
            return self.H_0(state)
        possible_moves = []
        neighbors = list(state.legal_moves)
        random.shuffle(neighbors)
        for neighbor in neighbors:
            # move simulation
            state.push(neighbor)
            minmax = minmax_heuristic.MyHeuristic()
            # assigning value to the move
            neighbor_value = minmax.H_L(state, 1)
            possible_moves.append((neighbor, neighbor_value))
            state.pop()
        # sorting
        possible_moves = sorted(possible_moves, key=lambda x: (x[1]))
        filtered_possible_moves = []
        if len(possible_moves) > 0:
            if state.turn:
                for i in range(0, 2):
                    if len(possible_moves) > 0:
                        filtered_possible_moves.append(possible_moves.pop())
            else:
                filtered_possible_moves = possible_moves[0:2]
            possible_moves = filtered_possible_moves
            best_move = possible_moves.pop()[0]
            state.push(best_move)
            best_value = self.H_L(state, l - 1)
            if l == 4 and best_value != 10000 and best_value != -10000:
                MyBestHeuristic.write(state, best_value)
            state.pop()
            while len(possible_moves) > 0:
                next_move = possible_moves.pop()[0]
                state.push(next_move)
                next_value = self.H_L(state, l - 1)
                if l == 4 and next_value != 10000 and next_value != -10000:
                    MyBestHeuristic.write(state, next_value)
                state.pop()
                if next_value > best_value and state.turn:
                    best_value = next_value
                    best_move = next_move
                if next_value < best_value and not state.turn:
                    best_value = next_value
                    best_move = next_move
            self.move = best_move
            return best_value
        else:
            if state.turn:
                return -10000
            else:
                return 10000

    @staticmethod
    def matrix_to_int(board):
        l = [None] * 64
        for sq in chess.scan_reversed(board.occupied_co[chess.WHITE]):
            l[sq] = board.piece_type_at(sq)
        for sq in chess.scan_reversed(board.occupied_co[chess.BLACK]):
            l[sq] = -board.piece_type_at(sq)
        return [0 if v is None else v for v in l]

    @staticmethod
    def write(board, value):
        f = open("./ts.csv", "a")
        list = MyBestHeuristic.matrix_to_int(board)
        f.write("[")
        for item in list:
            f.write(f"{item} ")
        f.write("],")
        f.write(f"{value}\n")
        f.close()


In [2]:
"""
    Predictive MinMax
"""

import random

from sklearn.linear_model import LinearRegression
from typing import Any

import pandas as pandas
from chess import Board

from minmax_heuristic.MinMaxBest import MyBestHeuristic


class Model:
    model: Any

    def __init__(self):
        ds = pandas.read_csv("./ts.csv")
        X = []
        for single in ds["data"]:
            x = single.replace("[", "").replace("]", "").replace(" ", ",").split(",")
            x.pop()
            X.append([int(numeric_string) for numeric_string in x])
        y = []
        for single in ds["target"]:
            y.append(single)
        self.model = LinearRegression()
        self.model.fit(X, y)

    def predict(self, board):
        x = [MyBestHeuristic.matrix_to_int(board)]
        return int(self.model.predict(x))


class MyPredictiveHeuristic:
    move: Any
    model = Model()

    def __init__(self):
        pass

    def __repr__(self):
        pass

    # metric of evaluation
    def H_0(self, state: Board):
        return self.model.predict(state)

    # min max
    def H_L(self, state: Board, l: int):
        if l == 0:
            return self.H_0(state)
        possible_moves = list(state.legal_moves)
        random.shuffle(possible_moves)
        if len(possible_moves) > 0:
            best_move = possible_moves.pop()
            state.push(best_move)
            best_value = self.H_L(state, l - 1)
            state.pop()
            while len(possible_moves) > 0:
                next_move = possible_moves.pop()
                state.push(next_move)
                next_value = self.H_L(state, l - 1)
                state.pop()
                if next_value > best_value and state.turn:
                    best_value = next_value
                    best_move = next_move
                if next_value < best_value and not state.turn:
                    best_value = next_value
                    best_move = next_move
            self.move = best_move
            return best_value
        else:
            if state.turn:
                return -10000
            else:
                return 10000


In [3]:
"""
    This is the main driver file. It will be responsible for displaying the object.

    NOTES IMPORTANT: When the program is running, hover the mouse over the board
    because it sometimes crashes!!!!!
"""

import chess
from minmax_heuristic import MyHeuristic
from alphabeta_heuristic import MyHeuristic2
import pygame as p

from minmax_heuristic.MinMaxBest import MyBestHeuristic
from minmax_heuristic.PredictiveMinMax import MyPredictiveHeuristic

WIDTH = HEIGHT = 512
# dimensions of a chess board are 8x8
DIMENSION = 8
SQ_SIZE = HEIGHT // DIMENSION
# for animations later on
MAX_FPS = 15
IMAGES = {}


# Initialize a global dictionary of images. This will be called exactly once in the main
def loadImages():
    pieces = ["wP", "wR", "wN", "wB", "wQ", "wK", "bP", "bR", "bN", "bB", "bQ", "bK"]
    for piece in pieces:
        IMAGES[piece] = p.transform.scale(p.image.load("img/" + piece + ".png"), (SQ_SIZE, SQ_SIZE))
    # Note: can access an image by saying 'IMG['wP']'


# The main driver of the code. This will update the graphics.
def main():
    p.init()
    screen = p.display.set_mode((WIDTH, HEIGHT))
    clock = p.time.Clock()
    screen.fill(p.Color("white"))
    # only do this once, before the while loop
    loadImages()
    running = True
    gameOver = False
    board = chess.Board()
    minmax = MyHeuristic()
    alphabeta = MyHeuristic2()
    best_minmax = MyBestHeuristic()
    predictive_minmax = MyPredictiveHeuristic()
    while running:
        for e in p.event.get():
            if e.type == p.QUIT:
                running = False
            clock.tick(MAX_FPS)
            p.display.flip()
            if not gameOver:
                if board.turn:
                    predictive_minmax.H_L(board, 1)
                    board.push(predictive_minmax.move)
                else:
                    best_minmax.H_L(board, 2)
                    board.push(best_minmax.move)
                gameOver = board.is_game_over()
            drawGameState(screen, make_chess_matrix(board))


# Responsible for all the graphics within a current game state
def drawGameState(screen, board):
    # draw squares on the board
    drawBoard(screen)
    # add in piece highlighting or move suggestions (later)
    # draw pieces on top of those squares
    drawPieces(screen, board)


"""
    Draw squares on the board. The top left square is always light.
"""


def drawBoard(screen):
    colors = [p.Color("white"), p.Color("grey")]
    for r in range(DIMENSION):
        for c in range(DIMENSION):
            color = colors[((r + c) % 2)]
            p.draw.rect(screen, color, p.Rect(c * SQ_SIZE, r * SQ_SIZE, SQ_SIZE, SQ_SIZE))


"""
    Draw the pieces on the board.
"""


def drawPieces(screen, board):
    for r in range(DIMENSION):
        for c in range(DIMENSION):
            piece = board[r][c]
            if piece != ".":
                if piece.isupper():
                    piece = "w" + piece
                else:
                    piece = "b" + piece.upper()
                screen.blit(IMAGES[piece], p.Rect(c * SQ_SIZE, r * SQ_SIZE, SQ_SIZE, SQ_SIZE))


def make_chess_matrix(board):
    pgn = board.epd()
    foo = []
    pieces = pgn.split(" ", 1)[0]
    rows = pieces.split("/")
    for row in rows:
        foo2 = []
        for thing in row:
            if thing.isdigit():
                for i in range(0, int(thing)):
                    foo2.append('.')
            else:
                foo2.append(thing)
        foo.append(foo2)
    return foo


if __name__ == '__main__':
    main()


pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
